In [1]:
import cvxpy as cvx
from zipline.optimize import MaximizeAlpha, TargetWeights
from zipline.optimize.constraints import MaxGrossExposure

In [2]:
import numpy as np
import pandas as pd
from numpy.testing import assert_almost_equal

In [3]:
stocks = ['000001', '000002', '000003', '000004', '000005']
alphas = pd.Series([0.1, 0.2, 0.3, -0.1, -0.4], index=stocks)
weights = pd.Series([-0.2, -0.2, 0.5, 0.5, 0.4], index=stocks)

# 测试

In [4]:
from zipline.optimize.constraints import CannotHold

In [5]:
obj = MaximizeAlpha(alphas)

In [6]:
l_w = obj.long_w
s_w = obj.short_w
w = obj.w
l_w_s = obj.long_weights_series
s_w_s = obj.short_weights_series

In [8]:
cnthd = CannotHold(stocks[2:4])

In [9]:
constraints = [con for con in cnthd.gen_constraints(l_w, s_w, l_w_s, s_w_s)]

In [10]:
constraints += [cvx.norm(w, 1) <= 1.5, l_w <= 0.5]

In [11]:
prob = cvx.Problem(obj.objective, constraints)
prob.solve(verbose=True)


ECOS 2.0.4 - (C) embotech GmbH, Zurich Switzerland, 2012-15. Web: www.embotech.com/ECOS

It     pcost       dcost      gap   pres   dres    k/t    mu     step   sigma     IR    |   BT
 0  +3.125e-02  -5.169e+00  +4e+01  6e-01  5e-01  1e+00  2e+00    ---    ---    1  1  - |  -  - 
 1  -2.194e-01  -3.586e-01  +1e+01  7e-02  6e-02  7e-01  7e-01  0.8012  2e-01   0  0  0 |  0  0
 2  -7.578e-01  -8.575e-01  +6e+00  3e-02  4e-02  4e-01  3e-01  0.8154  3e-01   0  0  0 |  0  0
 3  -5.947e-01  -6.001e-01  +3e-01  1e-03  2e-03  2e-02  1e-02  0.9545  2e-03   0  0  0 |  0  0
 4  -5.999e-01  -6.000e-01  +3e-03  1e-05  2e-05  2e-04  2e-04  0.9890  1e-04   0  0  0 |  0  0
 5  -6.000e-01  -6.000e-01  +4e-05  1e-07  2e-07  2e-06  2e-06  0.9890  1e-04   1  0  0 |  0  0
 6  -6.000e-01  -6.000e-01  +4e-07  2e-09  2e-09  2e-08  2e-08  0.9890  1e-04   1  0  0 |  0  0
 7  -6.000e-01  -6.000e-01  +4e-09  2e-11  3e-11  2e-10  2e-10  0.9890  1e-04   1  0  0 |  0  0

OPTIMAL (within feastol=2.5e-11, reltol=7.4e-

0.5999999999161069

In [12]:
sum(l_w.value) - sum(s_w.value)

1.5000000000097775

In [13]:
obj.short_weights_value

000001   -0.0
000002    0.0
000003   -0.0
000004   -0.0
000005   -1.5
dtype: float64

In [14]:
obj.long_weights_value

000001    0.0
000002    0.0
000003    0.0
000004    0.0
000005    0.0
dtype: float64